# Tipper

This object can be used to store tipper (ZTEM) surveys - a natural-source geophysical method. Data are provided in the frequency-domain as point source measurements of tipper data.

The following example shows how to generate a tipper survey with associated data stored in `geoh5` format and accessible from [Geoscience ANALYST](https://mirageoscience.com/mining-industry-software/geoscience-analyst/).


![mtSurvey](./images/tipper_survey.png)

In [ ]:
import numpy as np

from geoh5py.objects import TipperBaseStations, TipperReceivers
from geoh5py.workspace import Workspace


# Create a new project
workspace = Workspace("my_project.geoh5")

# Define the pole locations
n_stations = 64
n_lines = 2
x_loc, y_loc = np.meshgrid(
    np.linspace(0, 60, n_stations), np.linspace(-20, 20.0, n_lines)
)
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Assign a line ID to the poles (vertices)
parts = np.kron(np.arange(n_lines), np.ones(n_stations)).astype("int")

# Create the survey from vertices
receivers = TipperReceivers.create(workspace, vertices=vertices, parts=parts)
base = TipperBaseStations.create(workspace, vertices=vertices)

We have so far created two seperate entities, one for the receiver locations and another for the base station(s). In order to finalize the survey, the association must be made between the two entities:

In [ ]:
receivers.base_station = base

or equivalently

In [ ]:
base.receivers = receivers

Only one of the two options above is needed. 

## Metadata

Along with the [TipperReceivers](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.tipper.TipperReceivers), the metadata contains all the necessary information to define the geophysical experiment.

In [ ]:
receivers.metadata

### Channels

List of frequencies at which the data are provided.

In [ ]:
receivers.channels = [30.0, 45.0, 90.0, 180.0, 360.0, 720.0]

### Input type

Generic label used in the `geoh5` standard for all EM survey entities. Restricted to `Rx and base station` in the case of a tipper survey.

### Property groups

List of [PropertyGroup](../api/geoh5py.groups.rst#module-geoh5py.groups.property_group)s defining the various data components (e.g. `Txz (real)`, `Tyz (imag)`, ...). It is not required to supply all components of the impedence tensor, but it is expected that each component contains a list of data channels of length and in the same order as the `Channels` (one `Data` per frequency).

The class method [add_components_data](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.add_components_data) can help users add data from nested dictionaries. Below is an example using four components:

In [ ]:
# Arbitrary data generator using sine functions
def data_fun(c, f):
    (c + 1.0) * (f + 1.0) * np.sin(f * np.pi * (x_loc * y_loc).ravel() / 400.0)

# Create a nested dictionary of component and frequency data.
data = {
    component: {
        f"{component}_{freq}": {"values": data_fun(cc, ff)}
        for ff, freq in enumerate(receivers.channels)
    }
    for cc, component in enumerate(
        [
            "Txz (real)",
            "Txz (imaginary)",
            "Tyz (real)",
            "Tyz (imaginary)",
        ]
    )
}

receivers.add_components_data(data)

Metadata are updated immediately to reflect the addition of components:

In [ ]:
receivers.metadata

Data channels associated with each component can be quickly accessed through the [BaseEMSurvey.components](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.components) property:

In [ ]:
receivers.components

### Receivers

Generic label used in the `geoh5` standard for EM survey to identify the [TipperReceivers](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.tipper.TipperReceivers) entity.

### Base stations

Generic label used in the `geoh5` standard for EM survey to identify the [TipperBaseStations](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.tipper.TipperBaseStations) entity.

### Survey type

Label identifier for `ZTEM` survey type.

### Unit

Units for frequency sampling of the data: `Hertz (Hz)`, `KiloHertz (kHz)`, `MegaHertz (MHz)` or `Gigahertz (GHz)`.

In [ ]:
workspace.finalize()